*ToDo*

02) pred_window und obs_window ausprobieren.
03) output nochmal genau anschauen.
04) was ist maskiert anschauen - und wie genau?
05) VRAM wenig ausgelastet. Batch size mal mit 320 ausprobieren, aber auch durchziehen.
06) Task pred: scheduler einfügen.
07) Analyse each var (sparsity)
08) loss per var/quality of varis loss (is this already ablation to forecast only one var)
09) Include sepsis definition
10) 

In [ ]:
import pickle
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from tqdm import tqdm
tqdm.pandas()
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import roc_auc_score, precision_recall_curve, auc

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchinfo import summary
from torch.optim.lr_scheduler import ReduceLROnPlateau

In [ ]:
# data settings
test_cond = 0

if test_cond == 1:
    data_path = '/home/mitarb/fracarolli/files/231113_STraTS_new/mimic_iii_preprocessed_new_z_err20092.pkl'
    sample_divisor = 100
    number_of_epochs = 20
else:
    data_path = '/home/mitarb/fracarolli/files/231113_STraTS_new/mimic_iii_preprocessed_new_z_err20092.pkl'
    sample_divisor = 1
    number_of_epochs = 1000

## Load forecast dataset into matrices.

In [ ]:
def inv_list(l, start=0):  # Create vind
    d = {}
    for i in range(len(l)):
        d[l[i]] = i+start
    return d


def f(x):
    mask   = [0 for i in range(V)]
    values = [0 for i in range(V)]
    #print('hi', mask , V)
    #mask, values = np.zeros(V), np.zeros(V)
    #print('hi', mask)
    for vv in x:  # tuple of ['vind','value']
        v = int(vv[0])-1  # shift index of vind
        mask[v] = 1
        values[v] = vv[1]  # get value
    return values+mask  # concat


def pad(x):
    if len(x) > 880:
        print(len(x))
    return x+[0]*(fore_max_len-len(x))

In [ ]:
pred_window_old = 12  # hours
obs_windows = range(0, 1, 1)

# Read data.
data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))
# Remove test patients.
print(data.head())
raise Exception
data = data.merge(oc[['ts_ind', 'SUBJECT_ID']], on='ts_ind', how='left')
test_sub = oc.loc[oc.ts_ind.isin(test_ind)].SUBJECT_ID.unique()
data = data.loc[~data.SUBJECT_ID.isin(test_sub)]
oc = oc.loc[~oc.SUBJECT_ID.isin(test_sub)]
data.drop(columns=['SUBJECT_ID', 'TABLE'], inplace=True)
# Fix age.
data.loc[(data.variable == 'Age') & (data.value > 200), 'value'] = 91.4
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]  # static data are 'Age' and 'Gender'
data = data.loc[~ii]  # ~ binary flip
# print('data\n',data)

static_var_to_ind = inv_list(static_varis)  # {'Age': 0, 'Gender': 1}
D = len(static_varis)  # 2 by definition
N = data.ts_ind.max()+1  # /= 12: 52861
demo = np.zeros((int(N), int(D)))
for row in tqdm(static_data.itertuples()):
    demo[int(row.ts_ind), static_var_to_ind[row.variable]] = row.value
# print('Demo after tqdm command \n',demo[:10])
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)  # quite sparse
stds = demo.std(axis=0, keepdims=True)
stds = (stds == 0)*1 + (stds != 0)*stds
demo = (demo-means)/stds
# print('Demo after normalisation \n',demo[:10])
# Get variable indices.
varis = sorted(list(set(data.variable)))
V = len(varis)  # 129 for \=12 with varis all variables except for static ones
# print('varis', varis, V)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# print('data vind\n', data['vind'], '\n data\n',data)
# Find max_len.
fore_max_len = 880  # hard coded max_len of vars
# Get forecast inputs and outputs.
fore_times_ip = []
fore_values_ip = []
fore_varis_ip = []
fore_op = []
fore_op_awesome = []
fore_inds = []
for w in tqdm(range(25, 124, 4)):  # range(20, 124, 4), pred_window=2
    pred_data = data.loc[(data.hour>=w)&(data.hour<=w+24)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
    pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
    pred_data['vind_value'] = pred_data['vind_value'].apply(f)   

    obs_data = data.loc[(data.hour < w) & (data.hour >= w-24)]
    obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
    obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
    obs_data = obs_data.groupby('ts_ind').agg({'vind': list, 'hour': list, 'value': list}).reset_index()
    #Michi: Hier hole ich mir die 24 Stunden vor, und 24 Stunden nach einem bestimmten Zeitpunkt raus
    for pred_window  in range(-24, 24, 1):
        pred_data = data.loc[(data.hour >= w+pred_window) & (data.hour <= w+1 +pred_window)]
        pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value': 'first'}).reset_index()
        pred_data['vind_value'+str(pred_window)] = pred_data[['vind', 'value']].values.tolist()
        pred_data = pred_data.groupby('ts_ind').agg({'vind_value'+str(pred_window): list}).reset_index()
        pred_data['vind_value'+str(pred_window)] = pred_data['vind_value'+str(pred_window)].apply(f)  # 721 entries with 2*129 vind_values
        obs_data = obs_data.merge(pred_data, on='ts_ind')

    for col in ['vind', 'hour', 'value']:
        obs_data[col] = obs_data[col].apply(pad)
    fore_op_awesome.append(np.array(list([list(obs_data['vind_value'+str(pred_window)]) for pred_window in range(0, 24, 1)])))
    #fore_op.append(np.array(list(obs_data.vind_value)))
    fore_inds.append(np.array([int(x) for x in list(obs_data.ts_ind)]))
    fore_times_ip.append(np.array(list(obs_data.hour)))
    fore_values_ip.append(np.array(list(obs_data.value)))
    fore_varis_ip.append(np.array(list(obs_data.vind)))
    
del data
fore_times_ip = np.concatenate(fore_times_ip, axis=0)
fore_values_ip = np.concatenate(fore_values_ip, axis=0)
fore_varis_ip = np.concatenate(fore_varis_ip, axis=0)

fore_op_awesome = np.concatenate(fore_op_awesome, axis=1)
fore_op = np.swapaxes(fore_op_awesome, 0, 1)
print(fore_op.shape)

#raise Exception
fore_inds = np.concatenate(fore_inds, axis=0)
fore_demo = demo[fore_inds]
# Get train and valid ts_ind for forecast task.
train_sub = oc.loc[oc.ts_ind.isin(train_ind)].SUBJECT_ID.unique()
valid_sub = oc.loc[oc.ts_ind.isin(valid_ind)].SUBJECT_ID.unique()
rem_sub = oc.loc[~oc.SUBJECT_ID.isin(np.concatenate((train_ind, valid_ind)))].SUBJECT_ID.unique()
bp = int(0.8*len(rem_sub))
train_sub = np.concatenate((train_sub, rem_sub[:bp]))
valid_sub = np.concatenate((valid_sub, rem_sub[bp:]))
train_ind = oc.loc[oc.SUBJECT_ID.isin(train_sub)].ts_ind.unique()  # Add remaining ts_ind s of train subjects.
valid_ind = oc.loc[oc.SUBJECT_ID.isin(valid_sub)].ts_ind.unique()  # Add remaining ts_ind s of train subjects.
# Generate 3 sets of inputs and outputs.
train_ind = np.argwhere(np.in1d(fore_inds, train_ind)).flatten()
valid_ind = np.argwhere(np.in1d(fore_inds, valid_ind)).flatten()
fore_train_ip = [ip[train_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
fore_valid_ip = [ip[valid_ind] for ip in [fore_demo, fore_times_ip, fore_values_ip, fore_varis_ip]]
del fore_times_ip, fore_values_ip, fore_varis_ip, demo, fore_demo
fore_train_op = fore_op[train_ind]
fore_valid_op = fore_op[valid_ind]
del fore_op

print('lengths of rem_sub, fore_train_ip[1], fore_valid_ip[0]')
print(len(rem_sub), fore_train_ip[1].shape, fore_valid_ip[0].shape)

## Load target dataset into matrices.

In [ ]:
def get_sofa(matrix): #24x131 matrix
    # GCS: min_eye, min_motor, min_verbal = 5, 5, 5
    print(matrix.size())
    #raise Exception
    key ="GCS_eye"
    a=matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]
    print(a)
    min_eye = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    #print("calculatedvalue", min_eye)
    #print("table_value", matrix[:, var_to_ind[key]])
    #print("std", mean_std_dict[key][1])
    #print("mean", mean_std_dict[key][0])
    key = "GCS_motor"
    min_motor = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    key = "GCS_verbal"
    min_verbal = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=5)
    

    GCS = min_eye + min_motor + min_verbal
    if GCS > 14: GCS_sofa = 0
    elif GCS > 12: GCS_sofa = 1
    elif GCS > 9:  GCS_sofa = 2
    elif GCS > 5:  GCS_sofa = 3
    else: GCS_sofa = 4
    print('GCS_sofa is', GCS_sofa, ';     GCS is', GCS,'; GCS eye', min_eye, '; GCS motor', min_motor, '; GCS verbal', min_verbal)

    # bilirubin Liver SOFA

    key = "Bilirubin (Total)"
    bilir = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    print("calculatedvalue", bilir)
    print("table_value", matrix[:, var_to_ind[key]])
    print("std", mean_std_dict[key][1])
    print("mean", mean_std_dict[key][0])
    if bilir > 12: bilir_sofa = 4
    elif bilir > 6: bilir_sofa = 3
    elif bilir > 2: bilir_sofa = 2
    elif bilir > 1.2: bilir_sofa = 1
    else: bilir_sofa = 0
    print('bilir_Sofa is', bilir_sofa, ';   bilirubin is', bilir)
    
    # Coagulation (Platelets)
    key = "Platelet Count"
    plate = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=160)
    if plate > 150: plate_sofa = 0
    elif plate > 100: plate_sofa = 1
    elif plate > 50: plate_sofa = 2
    elif plate > 20: plate_sofa = 4
    else: plate_sofa = 4
    print('plate_sofa is', plate_sofa, ';   platelet count is', plate)
    
    # print('Urinmenge 24h', sum(data_var[data_var['variable']=='Urine']['value2']))

    key = "Urine"
    urine = sum(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key = "Creatinine Blood"
    creat = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    
    if (urine < 200) or (creat > 5): renal_sofa = 4
    elif  (urine < 500) or (creat > 3.5): renal_sofa = 3
    elif creat > 2.0: renal_sofa = 2
    elif creat > 1.2: renal_sofa = 1
    else: renal_sofa = 0
    print('renal_sofa:',renal_sofa,';       urine 24:',urine,'; creat:', creat)
    
    CS_data = get_CS(matrix)
    cs_sofa = CS_SOFA(CS_data)
    
    #cs_sofa = 0
    key="FiO2"
    fio2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    key="PO2"
    po2 = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])
    PaO2FiO2 = 100*po2/fio2
    print("size", PaO2FiO2.size())
    PaO2FiO2 = PaO2FiO2[torch.nonzero(PaO2FiO2, as_tuple=True)]
    pao2fio2 = min(PaO2FiO2)
    if pao2fio2<100: resp=4
    elif pao2fio2<200: resp=3
    elif pao2fio2<300:resp=2
    elif pao2fio2<400:resp=1
    else: resp=0
    return GCS_sofa, cs_sofa, resp, plate_sofa, bilir_sofa, renal_sofa

def get_CS(matrix):
    #data_var = data_pat[data_pat['variable'].isin(['Dobutamine','Dopamine','Epinephrine','Norepinephrine','Weight'])]
    #data_var['value2'] = data_var['value']*data_var['std']+data_var['mean']
    
    #weight = min(data_var[data_var['variable']=='Weight']['value2'], default=80)  # set default weight to 80kg.
    key = "Weight"
    weight = min(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)], default=80)#*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
    key = "Dopamine"
    try:
        data_dop = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dop = data_dop /60/weight*1000
    except:
        data_dop = 0

    key = "Dobutamine"
    
    try:
        data_dobu = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_dobu = data_dop /60/weight*1000
    except:
        data_dobu = 0
    key = "Epinephrine"
    
    try:
        data_epi = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_epi = data_dop /60/weight*1000
    except:
        data_epi = 0
    key = "Norepinephrine"
    
    try:
        data_nore = max(matrix[:, var_to_ind[key]][torch.nonzero(matrix[:, var_to_ind[key]], as_tuple=True)]*mean_std_dict[key][1]+mean_std_dict[key][0], default=1)
        data_nore = data_dop /60/weight*1000
    except:
        data_nore = 0
        


    key = "SBP"
    SBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    key = "DBP"
    DBP = (matrix[:, var_to_ind[key]]*mean_std_dict[key][1]+mean_std_dict[key][0])

    MAP = 2/3 * DBP + 1/3 * SBP
    MAP = min(MAP[torch.nonzero(MAP, as_tuple=True)], default=100)
                 
        
    return MAP, data_dop, data_dobu, data_epi, data_nore 
    
def CS_SOFA(data):
    map = data[0]
    dop, dobu, epi, nore = data[1:5]
    # print('CS data: mdden', data)
    if (dop > 15) or (epi > 0.1) or (nore > 0.01): CS = 4
    elif (dop > 5) or (epi > 0) or (nore > 0): CS = 3
    elif (dop > 0) or (dobu > 0): CS = 2
    elif map < 70: CS = 1
    else: CS = 0
    # print('CS Sofa is:', CS)
    return CS 
    


In [ ]:


fore_max_len = 880
# Read data.
# data_path = '/home/mitarb/fracarolli/files/230613_STraTS_preprocessed/mimic_iii_preprocessed.pkl'
data, oc, train_ind, valid_ind, test_ind = pickle.load(open(data_path, 'rb'))
means_stds = data.groupby("variable").agg({"mean":"first", "std":"first"})
#print(means_stds)
mean_std_dict = dict()
print(means_stds.keys)
for pos, row in means_stds.iterrows():
    print(pos)
    print(row)
    mean_std_dict[pos] = (float(row["mean"]), float(row["std"]))
print(mean_std_dict)
#raise Exception
# Filter labeled data in first 24h.
data = data.loc[data.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
data = data.loc[(data.hour >= 0) & (data.hour <= 48)]
old_oc = oc
oc = oc.loc[oc.ts_ind.isin(np.concatenate((train_ind, valid_ind, test_ind), axis=-1))]
# Fix age.
data.loc[(data.variable == 'Age') & (data.value > 200), 'value'] = 91.4
# Get y and N.
y = np.array(oc.sort_values(by='ts_ind')['in_hospital_mortality']).astype('float32')
print(y.shape)
#raise Exception
data['ts_ind'] = data['ts_ind'].astype(int) 
N = int(data.ts_ind.max() + 1)

# Create demographic/static data
# Get static data with mean fill and missingness indicator.
static_varis = ['Age', 'Gender']
ii = data.variable.isin(static_varis)
static_data = data.loc[ii]
data = data.loc[~ii]  # reduce data to non-demo/non-static
static_var_to_ind = inv_list(static_varis)
D = len(static_varis)
demo = np.zeros((int(N), D))
for row in tqdm(static_data.itertuples()):
    demo[int(row.ts_ind), static_var_to_ind[row.variable]] = row.value
# Normalize static data.
means = demo.mean(axis=0, keepdims=True)
stds = demo.std(axis=0, keepdims=True)
stds = (stds == 0)*1 + (stds != 0)*stds
demo = (demo-means)/stds

# Trim to max len.
data = data.sample(frac=1)
data = data.groupby('ts_ind').head(880)

# Get N, V, var_to_ind.
#N = data.ts_ind.max() + 1
varis = sorted(list(set(data.variable)))
# V = len(varis)
var_to_ind = inv_list(varis, start=1)
data['vind'] = data.variable.map(var_to_ind)
data = data[['ts_ind', 'vind', 'hour', 'value']].sort_values(by=['ts_ind', 'vind', 'hour'])
# Add obs index.
data = data.sort_values(by=['ts_ind']).reset_index(drop=True)
data = data.reset_index().rename(columns={'index': 'obs_ind'})
data = data.merge(data.groupby('ts_ind').agg({'obs_ind': 'min'}).reset_index().rename(columns={
                                                            'obs_ind': 'first_obs_ind'}), on='ts_ind')
data['obs_ind'] = data['obs_ind'] - data['first_obs_ind']
# Find max_len.
max_len = data.obs_ind.max()+1
print('max_len', max_len)
# Generate times_ip and values_ip matrices.
times_inp = np.zeros((N, max_len), dtype='float32')
values_inp = np.zeros((N, max_len), dtype='float32')
varis_inp = np.zeros((N, max_len), dtype='int32')
for row in tqdm(data.itertuples()):
    ts_ind = row.ts_ind
    l = row.obs_ind
    times_inp[ts_ind, l] = row.hour
    values_inp[ts_ind, l] = row.value
    varis_inp[ts_ind, l] = row.vind

w=0

fore_in = []

pred_data = data.loc[(data.hour>=0)&(data.hour<=48)]
pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value':'first'}).reset_index()
pred_data['vind_value'] = pred_data[['vind', 'value']].values.tolist()
pred_data = pred_data.groupby('ts_ind').agg({'vind_value':list}).reset_index()
pred_data['vind_value'] = pred_data['vind_value'].apply(f)   

obs_data = data.loc[(data.hour < 24) & (data.hour >= 0)]
resultdict = dict()
for ts_ind in obs_data.ts_ind:
    resultdict[ts_ind] = [[]]
obs_data = obs_data.loc[obs_data.ts_ind.isin(pred_data.ts_ind)]
obs_data = obs_data.groupby('ts_ind').head(fore_max_len)
obs_data = obs_data.groupby('ts_ind').agg({'vind': list, 'hour': list, 'value': list}).reset_index()

for pred_window  in range(24, 48, 1):
    print(pred_window)
    #print(w+1 +pred_window)
    #print(w+pred_window)
    pred_data = data.loc[(data.hour >= w+pred_window) & (data.hour <= w+1 +pred_window)]
    pred_data = pred_data.groupby(['ts_ind', 'vind']).agg({'value': 'first'}).reset_index()
    pred_data['vind_value'+str(pred_window)] = pred_data[['vind', 'value']].values.tolist()
    pred_data = pred_data.groupby('ts_ind').agg({'vind_value'+str(pred_window): list}).reset_index()
    pred_data['vind_value'+str(pred_window)] = pred_data['vind_value'+str(pred_window)].apply(f)  # 721 entries with 2*129 vind_values
    obs_data = obs_data.merge(pred_data, on='ts_ind')
    #print(list(data['vind_value'+str(pred_window)]))
    #print(len(list(data['vind_value'+str(pred_window)])))
    #print(len(list(data['vind_value'+str(pred_window)])[0]))

blub = (np.array(list([list(obs_data['vind_value'+str(pred_window)]) for pred_window in range(24, 48, 1)])))
print("bleb", blub.shape)
op = np.swapaxes(blub, 0, 1)

weird_oc = oc.loc[oc.ts_ind.isin(obs_data.ts_ind)]
y = np.array(weird_oc.sort_values(by='ts_ind')['in_hospital_mortality']).astype('float32')
print(y.shape)
data.drop(columns=['obs_ind', 'first_obs_ind'], inplace=True)

train_ind = [x for x in train_ind if x < op.shape[0]]
valid_ind = [x for x in valid_ind if x < op.shape[0]]
test_ind = [x for x in test_ind if x < op.shape[0]]

train_input = op[train_ind]
valid_input = op[valid_ind]
test_input = op[test_ind]

# Generate 3 sets of inputs and outputs.
train_ip = [ip[train_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
valid_ip = [ip[valid_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
test_ip = [ip[test_ind] for ip in [demo, times_inp, values_inp, varis_inp]]
del demo, times_inp, values_inp, varis_inp  # warum wird demo nicht gelöscht?

if test_cond == 1:
    tr_ind = [divmod(tr, 12)[0] for tr in train_ind]
    va_ind = [divmod(tr, 12)[0] for tr in valid_ind]
    te_ind = [divmod(tr, 12)[0] for tr in test_ind]
    
    train_op = y[tr_ind]  # is a problem for the test case...
    valid_op = y[va_ind]
    test_op = y[te_ind]
else:
    train_op = y[train_ind]  # is a problem for the test case...
    valid_op = y[valid_ind]
    test_op = y[test_ind]
print('y is:', y)
del y

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")


## Pretrain on forecasting.

In [ ]:
lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_detach.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception

        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=False, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p, "mean and std", np.mean(loss_list), np.std(loss_list))
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')

#STRATS, IMS, Sampling, Detach

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_backprop.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception

        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=False, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')
#STRATS, IMS, Sampling, No Detach

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_tf.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception

        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')


#STRATS, IMS, Teacher Forcing

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=False):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda()):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_dms.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception

        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')
#STRATS, DMS

# Scheduled Sampling DD

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        if trainn and schedule:
            
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output.detach(), linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")


lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_detach_schedule.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=True, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p, "mean and std", np.mean(loss_list), np.std(loss_list))
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')

#STRATS, IMS, Sampling, Detach

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        if trainn and schedule:
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output, linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_backprop_schedule.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=True, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')
#STRATS, IMS, Sampling, No Detach

# Scheduled Sampling DI

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        if trainn and schedule:
            
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output.detach(), linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_detach_schedule_dtf.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        prob = max([0.25, 0.25+(1.0-0.25)*(1-e/200)]) #DTF
        
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=True, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p, "mean and std", np.mean(loss_list), np.std(loss_list))
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')

#STRATS, IMS, Sampling, Detach

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        if trainn and schedule:
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output, linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_backprop_schedule_dtf.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        prob = max([0.25, 0.25+(1.0-0.25)*(1-e/200)]) #DTF
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=True, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')
#STRATS, IMS, Sampling, No Detach

# Scheduled Sampling RI

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        if trainn and schedule:
            
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output.detach(), linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_detach_schedule_random.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=False, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p, "mean and std", np.mean(loss_list), np.std(loss_list))
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')

#STRATS, IMS, Sampling, Detach

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        if trainn and schedule:
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output, linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_backprop_schedule_random.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=False, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')
#STRATS, IMS, Sampling, No Detach

# Scheduled Sampling RD

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        
        if trainn and schedule:
            
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output.detach(), linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output.detach(), linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")


lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_detach_schedule_random_dtf.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        prob = max([0.25, 0.25+(1.0-0.25)*(1-e/200)]) #DTF
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=False, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p, "mean and std", np.mean(loss_list), np.std(loss_list))
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')

#STRATS, IMS, Sampling, Detach

In [ ]:
class CVE(nn.Module):
    def __init__(self, hid_units, output_dim):
        super(CVE, self).__init__()
        self.hid_units = hid_units
        self.output_dim = output_dim
        self.W1 = nn.Parameter(torch.Tensor(1, self.hid_units))
        self.b1 = nn.Parameter(torch.Tensor(self.hid_units))
        self.W2 = nn.Parameter(torch.Tensor(self.hid_units, self.output_dim))
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)

    def forward(self, x):
        x = x.unsqueeze(-1)
        x = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W1), self.b1)), self.W2)
        return x


class Attention(nn.Module):
    def __init__(self, hid_dim):
        super(Attention, self).__init__()
        self.hid_dim = hid_dim
        self.W = nn.Parameter(torch.Tensor(50, self.hid_dim))
        self.b = nn.Parameter(torch.Tensor(self.hid_dim))
        self.u = nn.Parameter(torch.Tensor(self.hid_dim, 1))
        nn.init.xavier_uniform_(self.W)
        nn.init.zeros_(self.b)
        nn.init.xavier_uniform_(self.u)

    def forward(self, x, mask, mask_value=-1e30):
        attn_weights = torch.matmul(torch.tanh(torch.add(torch.matmul(x, self.W), self.b)), self.u)
        mask = mask.unsqueeze(-1)
        attn_weights = mask * attn_weights + (1 - mask) * mask_value
        attn_weights = F.softmax(attn_weights, dim=-2)
        return attn_weights


class Transformer(nn.Module):
    def __init__(self, N=2, h=8, dk=None, dv=None, dff=None, dropout=0, d=8):
        super(Transformer, self).__init__()
        self.N, self.h, self.dk, self.dv, self.dff, self.dropout = N, h, dk, dv, dff, dropout
        self.epsilon = torch.finfo(torch.float32).eps * torch.finfo(torch.float32).eps
        self.Wq = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wk = nn.Parameter(torch.Tensor(N, h, d, dk))
        self.Wv = nn.Parameter(torch.Tensor(N, h, d, dv))
        self.Wo = nn.Parameter(torch.Tensor(N, dv * h, d))
        self.W1 = nn.Parameter(torch.Tensor(N, d, dff))
        self.b1 = nn.Parameter(torch.Tensor(N, dff))
        self.W2 = nn.Parameter(torch.Tensor(N, dff, d))
        self.b2 = nn.Parameter(torch.Tensor(N, d))
        self.gamma = nn.Parameter(torch.Tensor(2 * N))
        self.beta = nn.Parameter(torch.Tensor(2 * N))
        nn.init.xavier_uniform_(self.Wq)
        nn.init.xavier_uniform_(self.Wk)
        nn.init.xavier_uniform_(self.Wv)
        nn.init.xavier_uniform_(self.Wo)
        nn.init.xavier_uniform_(self.W1)
        nn.init.zeros_(self.b1)
        nn.init.xavier_uniform_(self.W2)
        nn.init.zeros_(self.b2)
        nn.init.ones_(self.gamma)
        nn.init.zeros_(self.beta)
        self.training = False
        
    def forward(self, x, mask, mask_value=-1e-30):
        mask = mask.unsqueeze(-2)
        for i in range(self.N):
            # MHA
            mha_ops = []
            for j in range(self.h):
                q = torch.matmul(x, self.Wq[i, j, :, :])          
                k = torch.matmul(x, self.Wk[i, j, :, :]).permute(0, 2, 1)
                v = torch.matmul(x, self.Wv[i, j, :, :])
                A = torch.matmul(q, k)
                # Mask unobserved steps.
                A = mask * A + (1 - mask) * mask_value
                # Mask for attention dropout.
                if self.training:
                    dp_mask = (torch.rand(A.shape, device=A.device) >= self.dropout).float()
                    A = A * dp_mask + (1 - dp_mask) * mask_value
                A = F.softmax(A, dim=-1)
                mha_ops.append(torch.matmul(A, v))
            conc = torch.cat(mha_ops, dim=-1)
            proj = torch.matmul(conc, self.Wo[i, :, :])
            # Dropout.
            if self.training:
                proj = F.dropout(proj, p=self.dropout)
            # Add & LN
            x = x + proj
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i] + self.beta[2 * i]
            # FFN
            ffn_op = torch.add(torch.matmul(F.relu(torch.add(
                torch.matmul(x, self.W1[i, :, :]), self.b1[i, :])),
                                            self.W2[i, :, :]), self.b2[i, :])
            # Dropout.
            if self.training:
                ffn_op = F.dropout(ffn_op, p=self.dropout)
            # Add & LN
            x = x + ffn_op
            mean = x.mean(dim=-1, keepdim=True)
            variance = torch.mean(torch.square(x - mean), dim=-1, keepdim=True)
            std = torch.sqrt(variance + self.epsilon)
            x = (x - mean) / std
            x = x * self.gamma[2 * i + 1] + self.beta[2 * i + 1]
        return x


class StratsModel(nn.Module):
    def __init__(self, D, max_len, V, d, N, he, dropout, forecast=False, autoregressive=True):
        super(StratsModel, self).__init__()
        self.forecast = forecast
        self.autoregressive=autoregressive
        self.demo_enc = nn.Sequential(nn.Linear(D, 2 * d),
                                      nn.Tanh(),
                                      nn.Linear(2 * d, d),
                                      nn.Tanh())
        self.varis_emb = nn.Embedding(V + 1, d)
        nn.init.uniform_(self.varis_emb.weight, a=-0.05, b=0.05)
        self.values_emb = CVE(int(np.sqrt(d)), d)
        self.times_emb = CVE(int(np.sqrt(d)), d)
        # self.comb_emb = nn.LayerNorm(d)
        self.transformer = Transformer(N, he, dk=d//he, dv=d//he, dff=2*d, dropout=dropout, d=d)

        if autoregressive:
            self.linearize = nn.Linear(50, 131)
            self.dec = nn.TransformerDecoderLayer(d_model=131, dim_feedforward=131, nhead=1, batch_first=True)
        else:
            self.attention = Attention(2 * d)
            self.attentions = nn.ModuleList([Attention(2*d) for i in range(24)])
            # self.concat = nn.Linear(2 * d, 1)
            self.fore_op_calc = nn.Linear(2 * d, V)
            self.op_calc = nn.Linear(V, 1)
            self.sig = nn.Sigmoid()        
        

    def forward(self, demo, times, values, varis, trainn=False, tgt=torch.zeros(32, 1, 131).cuda(), schedule=True, probability=0.5, deterministic=False):
        demo_enc = self.demo_enc(demo)
        varis_emb = self.varis_emb(varis)
        values_emb = self.values_emb(values)
        times_emb = self.times_emb(times)

        comb_emb = (torch.sum(torch.stack([varis_emb, values_emb, times_emb]), dim=0))
        self.trainn = trainn
        mask = torch.clip(varis, 0, 1)
        cont_emb = self.transformer(comb_emb, mask)
        
        if trainn and schedule:
            output = torch.zeros(cont_emb.size(0), 1, 131).cuda()
            output.requires_grad=True
            linear_memory  = self.linearize(cont_emb)
            for i in range(24):
                #print(i)
                #print(linear_memory.size(), output.size())
                res = self.dec(output, linear_memory)
                if deterministic:
                    sample = i/24
                else:
                    sample = torch.rand(1).item()
                #print(res.size(), output.size())
                print(sample, probability)
                if sample < probability:
                
                    
                    output = torch.concat([output, tgt[:,i:i+1,:]], dim=1)
                else:
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
            return output[:, 1:, :] 
        
        if self.autoregressive:
            if self.trainn:
                #print(cont_emb.size())
                
                mask_new = torch.nn.Transformer.generate_square_subsequent_mask(25).cuda()
                linear_memory = self.linearize(cont_emb)
                start_input = torch.zeros(32, 1, 131).cuda()
                tgt = torch.concat([start_input, tgt], dim=1)
                result=self.dec(tgt, linear_memory, mask_new)
                return result[:, 1:, :]
            if not self.trainn:
                output = torch.zeros(32, 1, 131).cuda()
                linear_memory  = self.linearize(cont_emb)
                for i in range(24):
                    #print(i)
                    #print(linear_memory.size(), output.size())
                    res = self.dec(output, linear_memory)
                    #print(res.size(), output.size())
                    output = torch.concat([output, res[:,-1:,:]], dim=1)
                return output[:, 1:, :]
                
        if not self.autoregressive:
            fused_embs = []
            for attention in self.attentions:
                attn_weights = attention(cont_emb, mask)
                fused_emb = torch.matmul(attn_weights.transpose(-1, -2), cont_emb).squeeze(-2)
                # fused_emb torch.Size([32, 50])
                fused_embs.append(fused_emb)
            # fused_embs Liste mit 6 torch.Size([32, 50])
            fused_emb = torch.stack(fused_embs, axis=1)
            # altes fused_emb: torch.Size([32, 50])
            # neues fused_emb: torch.Size([32, 6, 50])
            # demo_enc torch.Size([32, 50])

            # demo_enc.unsqueeze(axis=-2).repeat(1, 6, 1):  torch.Size([32, 6, 50])
            #print(fused_emb.size())
            concat = torch.cat([fused_emb, demo_enc.unsqueeze(axis=-2).repeat(1, 24, 1)], axis=-1)
            #concat torch.Size([32, 6, 100])

            x = self.fore_op_calc(concat)
            # neues x torch.Size([32, 6, 258])
            # altes x: torch.Size([32, 258])
            if self.forecast:
                return x
            else:
                x = self.sig(self.op_calc(x)).mean(axis=-2)
                return x

print("Model definitions loaded")

lr, batch_size, samples_per_epoch, patience = 0.0005, 32, int(102400/sample_divisor), 6
d, N, he, dropout = 50, 2, 4, 0.2
print('number of parameters: ', V)

model = StratsModel(2, fore_max_len, V, d, N, he, dropout, forecast=True).cuda()
a = summary(model, [(32, 2), (32, 880), (32, 880), (32, 880)],  # shape of fore_train_ip
            dtypes=[torch.float, torch.float, torch.float, torch.long])
print(a)  # Model summary
# raise Exception
optimizer = torch.optim.Adam(model.parameters(), lr=lr)
scheduler = ReduceLROnPlateau(optimizer, 'min', patience=3)
# Pretrain fore_model.
best_val_loss = np.inf
N_fore = len(fore_train_op)
#print(N_fore)
fore_savepath = 'mimic_iii_sparse_ims_sampling_backprop_schedule_random_dtf.pytorch'
loss_func = torch.nn.MSELoss(reduction="none")

# torch.compile(model)
for e in range(number_of_epochs):
    e_indices = np.random.choice(range(N_fore), size=samples_per_epoch, replace=False)
    e_loss = 0
    pbar = tqdm(range(0, len(e_indices), batch_size))
    model.train()
    for start in pbar:
        ind = e_indices[start:start+batch_size]
        # raise Exception
        prob=min([1.0, 1.0+(0.25-1.0)*(1-e/200)])
        prob = max([0.25, 0.25+(1.0-0.25)*(1-e/200)]) #DTF
        output = model(*[torch.tensor(ip[ind], dtype=torch.float32).cuda() if i < 3 
                         else torch.tensor(ip[ind], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_train_ip)], trainn=True, tgt=torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda(), schedule=True, deterministic=False, probability=prob)
        #raise Exception
        #print(fore_train_op[ind].shape)
        #raise Exception
        loss = torch.tensor(fore_train_op[ind, :, 131:], dtype=torch.float32).cuda()*(
            output-torch.tensor(fore_train_op[ind, :, :131], dtype=torch.float32).cuda())**2
        loss = loss.sum(axis=-1).mean()
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()
        e_loss += loss.detach()
        pbar.set_description('%f' % e_loss)
    val_loss = 0
    model.eval()
    loss_list = []
    pbar = tqdm(range(0, len(fore_valid_op)-32, 32))  # len(fore_valid_op)           ####################   maybe also batch_size instead of 32
    for start in pbar:
        #print(start, start+batch_size)
        output = model(*[torch.tensor(ip[start:start+batch_size], dtype=torch.float32).cuda() 
                         if i < 3 else torch.tensor(ip[start:start+batch_size], dtype=torch.int32).cuda() 
                         for i, ip in enumerate(fore_valid_ip)], trainn=False, tgt=torch.tensor(fore_valid_op[start:start+batch_size], dtype=torch.float32).cuda())
        loss = torch.tensor(fore_valid_op[start:start+batch_size,:, 131:], dtype=torch.float32).cuda()*loss_func(
            output, torch.tensor(fore_valid_op[start:start+batch_size,:, :131], dtype=torch.float32).cuda())
        loss_list.extend(loss.sum(axis=-1).mean(axis=-1).detach().cpu().tolist())
        loss = loss.sum(axis=-1).mean()
        val_loss += loss.detach().cpu()
        pbar.set_description('%f' % val_loss)
    loss_p = e_loss*batch_size/samples_per_epoch
    val_loss_p = val_loss*batch_size/len(fore_valid_op)
    print('Epoch', e, 'loss', loss_p, 'val loss', val_loss_p)
    with open('loss_values_log', 'a') as f:
        f.write(str(e)+' ' + str(loss_p.item()) + ' ' + str(val_loss_p.item())+ '\n')
    scheduler.step(val_loss_p.item())
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), fore_savepath)
        best_epoch = e
    if (e-best_epoch) > patience:
        break
print('Training has ended.')
#STRATS, IMS, Sampling, No Detach